In [ ]:
import pandas as pd
from darts import TimeSeries

In [ ]:
#df = pd.read_csv(r"C:\Users\jakob\Downloads\archive (1)\AirPassengers.csv")

ab_df = pd.read_csv("../../data\absences_aggregated.csv', sep=',')
ab_df

In [ ]:
ab_df = ab_df[['index', 'count']]
# drop all rows with index older than 2023-03-01
ab_df = ab_df.drop(ab_df[ab_df['index'] > '2023-03-01'].index)
# normalize data   
ab_df['count'] = ab_df['count'] / ab_df['count'].max()
ab_df


In [ ]:
#series = TimeSeries.from_dataframe(df, 'Month', '#Passengers')
#series = TimeSeries.from_dataframe(df, 'Month', '#Passengers')
#series
#creaet new dataframe with only date and count
ab_df['index'] = pd.to_datetime(ab_df['index'])

series = TimeSeries.from_dataframe(ab_df, 'index', 'count')
series

In [ ]:
from darts.models import ExponentialSmoothing

train, val = series.split_before(pd.Timestamp('20220701'))

model = ExponentialSmoothing()
model.fit(train)
prediction = model.predict(len(val))

In [ ]:
import matplotlib.pyplot as plt

series.plot(label='actual')
prediction.plot(label='forecast', lw=3)
plt.legend()


In [ ]:
from darts.models import Prophet

models = [ExponentialSmoothing(), Prophet()]

backtests = [model.historical_forecasts(series,
                            start=.7,
                            forecast_horizon=3)
             for model in models]

In [ ]:
from darts.metrics import mape

series.plot(label='data')
for i, m in enumerate(models):
    err = mape(backtests[i], series)
    backtests[i].plot(lw=3, label='{}, MAPE={:.2f}%'.format(m, err))

plt.title('Backtesting results')
plt.legend()